In [62]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
from bs4 import BeautifulSoup
import lxml

In [63]:
req=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [64]:
soup=BeautifulSoup(req,features='html.parser')

In [65]:
data=soup.find('table')

In [66]:
df=pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])

In [67]:
for table in data.find_all('tr'):
    row=[]
    for row_info in table.find_all('td'):
        row.append(row_info.text.strip())
    if len(row)==3:
        df.loc[len(df)] =row
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [68]:
df=df.drop(df[(df.Borough=="Not assigned")].index)

In [69]:
df.Neighborhood.replace("Not assigned",df.Borough,inplace=True)

In [71]:
df2=df.groupby('PostalCode')['Neighborhood'].apply(lambda x: ', '.join(x))
df2=df2.reset_index(drop=False)

In [73]:
df3=pd.merge(df,df2,on='PostalCode')

In [74]:
df3.drop_duplicates(subset='PostalCode',inplace=True)
df3.drop(['Neighborhood_x'],axis=1,inplace=True)
df3.rename(columns={'Neighborhood_y':'Neighborhood'},inplace=True)
df3=df3.reset_index(drop=False)

In [75]:
df3.drop(['index'],axis=1,inplace=True)
df3.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [76]:
geodf=pd.read_csv('http://cocl.us/Geospatial_data')
geodf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [77]:
geodf.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df_toronto= pd.merge(geodf,df3, on='PostalCode')

In [78]:
df_toronto=df_toronto[['PostalCode','Borough','Neighborhood','Latitude','Longitude']]
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [83]:
address = 'Toronto, ON, Canada'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [84]:
Toronto_data=df_toronto
Toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [85]:
map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Borough'], Toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

In [86]:
CLIENT_ID = 'id removed'
CLIENT_SECRET = 'secret removed'
VERSION = '20190512' 
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: id removed
CLIENT_SECRET:secret removed


In [87]:
LIMIT = 10
def getNearbyVenues(names, latitudes, longitudes, radius=500): 
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Borough'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [34]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,62,62,62,62,62,62
Downtown Toronto,184,184,184,184,184,184
East Toronto,44,44,44,44,44,44
East York,42,42,42,42,42,42
Etobicoke,66,66,66,66,66,66
Mississauga,10,10,10,10,10,10
North York,125,125,125,125,125,125
Scarborough,79,79,79,79,79,79
West Toronto,60,60,60,60,60,60


In [36]:
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot =Toronto_onehot[fixed_columns]
Toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Store,Bike Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Coffee Shop,College Gym,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Rental Car Location,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Scarborough,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Scarborough,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Scarborough,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Scarborough,0,0,0,0,0,0,0,0,0,0,0,0,

In [38]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Store,Bike Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Coffee Shop,College Gym,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Korean Restaurant,Lake,Latin American Restaurant,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Rental Car Location,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Central Toronto,0.016129,0.000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016129,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00000,0.000000,0.000,0.016129,0.000000,0.000,0.000000,0.016129,0.000000,0.032258,0.000000,0.000000,0.000000,0.000,0.048387,0.000000,0.000000,0.000000,0.016129,0.000,0.000000,0.016129,0.080645,0.000000,0.000000,0.000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.016129,0.048387,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016129,0.000000,0.000000,0.000000,0.000000,0.000,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016129,0.000000,0.000000,0.000000,0.000000,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.016129,0.000000,0.032258,0.000,0.000000,0.016129,0.000000,0.000000,0.016129,0.000000,0.016129,0.000000,0.016129,0.000000,0.000000,0.000,0.000,0.000000,0.000,0.016129,0.016129,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080645,0.000000,0.000000,0.000000,0.016129,0.016129,0.000000,0.016129,0.000000,0.000000,0.032258,0.000000,0.000000,0.048387,0.000000,0.000000,0.016129,0.016129,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.016129,0.000000,0.000000,0.016129,

In [61]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt North, L'Amoreaux East, Milliken, Steeles East, Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                 venue  freq
0           Playground   0.5
1                 Park   0.5
2  American Restaurant   0.0
3            Pet Store   0.0
4    Korean Restaurant   0.0


----Agincourt, Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1                     Lounge   0.2
2             Breakfast Spot   0.2
3               Skating Rink   0.2
4             Clothing Store   0.2


----Birch Cliff, Cliffside West, Birch Cliff, Cliffside West----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2                   Café  0.25
3  General Entertainment  0.25
4              Pet Store  0.00


----Cedarbrae, Cedarbrae----
                  venue  freq
0                Bakery  0.12
1                  Bank  0.12
2       Thai Restaurant  0.12
3    Athletics & Sports  0.12
4  Caribbean Restaurant  0.1

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] =Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

In [42]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Park,Coffee Shop,Café,Sushi Restaurant,Dessert Shop,Restaurant,Bus Line,Indian Restaurant,Pub,American Restaurant
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Park,Italian Restaurant,Steakhouse,Gym,Gastropub,Bakery,French Restaurant
2,East Toronto,Ice Cream Shop,Italian Restaurant,Greek Restaurant,Brewery,Pub,Burrito Place,Bookstore,Comic Shop,Sandwich Place,Restaurant
3,East York,Pharmacy,Gym,Pizza Place,Park,Coffee Shop,Indian Restaurant,Grocery Store,Fish & Chips Shop,Pet Store,Liquor Store
4,Etobicoke,Pizza Place,Sandwich Place,Coffee Shop,Grocery Store,Fast Food Restaurant,Liquor Store,Pharmacy,Café,Bakery,Convenience Store


In [44]:
Toronto_count3.head()

0           Scarborough
1           Scarborough
2           Scarborough
3           Scarborough
4           Scarborough
236        East Toronto
331    Downtown Toronto
580        West Toronto
596    Downtown Toronto
598         Mississauga
Name: Neighborhood, dtype: object

In [45]:
import matplotlib.pyplot as plt

fig,ax= plt.subplots()
ax.bar(['Scarborough','Downtown T','West T','Mississauga',' East Toronto'],[5,2,1,1,1])
plt.show();

<Figure size 640x480 with 1 Axes>

In [46]:
Toronto_data=df_toronto[df_toronto['Borough'].str.contains("Scarborough")].reset_index(drop=True)
Toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union, Highla...",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ...",43.763573,-79.188711
3,M1G,Scarborough,"Woburn, Woburn",43.770992,-79.216917
4,M1H,Scarborough,"Cedarbrae, Cedarbrae",43.773136,-79.239476


In [47]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

Rouge, Malvern, Rouge, Malvern
Highland Creek, Rouge Hill, Port Union, Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill, Guildwood, Morningside, West Hill
Woburn, Woburn
Cedarbrae, Cedarbrae
Scarborough Village, Scarborough Village
East Birchmount Park, Ionview, Kennedy Park, East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge, Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West, Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West, Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights, Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford, Maryvale, Wexford
Agincourt, Agincourt
Clarks Corners, Sullivan, Tam O'Shanter, Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East, Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, L'Amoreaux West
Upper Rouge, Upper Rouge


In [48]:
print(Toronto_venues.shape)
Toronto_venues.head()

(79, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern, Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern, Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union, Highla...",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Highland Creek, Rouge Hill, Port Union, Highla...",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Highland Creek, Rouge Hill, Port Union, Highla...",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target


In [49]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Agincourt North, L'Amoreaux East, Milliken, Steeles East, Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Agincourt, Agincourt",5,5,5,5,5,5
"Birch Cliff, Cliffside West, Birch Cliff, Cliffside West",4,4,4,4,4,4
"Cedarbrae, Cedarbrae",8,8,8,8,8,8
"Clairlea, Golden Mile, Oakridge, Clairlea, Golden Mile, Oakridge",9,9,9,9,9,9
"Clarks Corners, Sullivan, Tam O'Shanter, Clarks Corners, Sullivan, Tam O'Shanter",10,10,10,10,10,10
"Cliffcrest, Cliffside, Scarborough Village West, Cliffcrest, Cliffside, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Scarborough Town Centre, Wexford Heights, Dorset Park, Scarborough Town Centre, Wexford Heights",5,5,5,5,5,5
"East Birchmount Park, Ionview, Kennedy Park, East Birchmount Park, Ionview, Kennedy Park",4,4,4,4,4,4


In [51]:
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot =Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Construction & Landscaping,Department Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,"Rouge, Malvern, Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern, Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union, Highla...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Highland Creek, Rouge Hill, Port Union, Highla...",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Highland Creek, Rouge Hill, Port Union, Highla...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [52]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Construction & Landscaping,Department Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000,0.000,0.00,0.0,0.000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.500000,0.0,0.0,0.000000,0.5,0.0,0.000000,0.0,0.0,0.00,0.00,0.000000,0.000,0.0
1,"Agincourt, Agincourt",0.0,0.000,0.000000,0.000,0.000000,0.200000,0.000000,0.000000,0.00,0.000,0.00,0.2,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000,0.000,0.00,0.0,0.000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.2,0.2,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.20,0.00,0.000000,0.000,0.0
2,"Birch Cliff, Cliffside West, Birch Cliff, Clif...",0.0,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.25,0.000,0.00,0.0,0.000000,0.25,0.000000,0.00,0.000000,0.0,0.000,0.000,0.25,0.0,0.000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.25,0.00,0.000000,0.000,0.0
3,"Cedarbrae, Cedarbrae",0.0,0.125,0.125000,0.125,0.000000,0.000000,0.000000,0.000000,0.00,0.125,0.00,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.125,0.125,0.00,0.0,0.125,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.00,0.000000,0.125,0.0
4,"Clairlea, Golden Mile, Oakridge, Clairlea, Gol...",0.0,0.000,0.222222,0.000,0.000000,0.000000,0.222222,0.111111,0.00,0.000,0.00,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000,0.000,0.00,0.0,0.000,0.00,0.111111,0.0,0.111111,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.111111,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.00,0.111111,0.000,0.0
5,"Clarks Corners, Sullivan, Tam O'Shanter, Clark...",0.0,0.000,0.000000,0.100,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.10,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.1,0.100,0.100,0.00,0.0,0.000,0.00,0.000000,0.0,0.000000,0.1,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.1,0.000000,0.0,0.0,0.100000,0.0,0.0,0.000000,0.0,0.1,0.00,0.00,0.000000,0.100,0.0
6,"Cliffcrest, Cliffside, Scarborough Village Wes...",0.5,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000,0.000,0.00,0.0,0.000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.5,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.00,0.000000,0.000,0.0
7,"Dorset Park, Scarborough Town Centre, Wexford ...",0.0,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.20,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000,0.000,0.00,0.0,0.000,0.00,0.000000,0.4,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.2,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.00,0.000000,0.000,0.2
8,"East Birchmount Park, Ionview, Kennedy Park, E...",0.0,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.25,0.0,0.250000,0.00,0.000000,0.25,0.000000,0.0,0.000,0.000,0.00,0.0,0.000,0.25,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.

In [53]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt North, L'Amoreaux East, Milliken, Steeles East, Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                 venue  freq
0           Playground   0.5
1                 Park   0.5
2  American Restaurant   0.0
3            Pet Store   0.0
4    Korean Restaurant   0.0


----Agincourt, Agincourt----
                       venue  freq
0  Latin American Restaurant   0.2
1                     Lounge   0.2
2             Breakfast Spot   0.2
3               Skating Rink   0.2
4             Clothing Store   0.2


----Birch Cliff, Cliffside West, Birch Cliff, Cliffside West----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2                   Café  0.25
3  General Entertainment  0.25
4              Pet Store  0.00


----Cedarbrae, Cedarbrae----
                  venue  freq
0                Bakery  0.12
1                  Bank  0.12
2       Thai Restaurant  0.12
3    Athletics & Sports  0.12
4  Caribbean Restaurant  0.1

In [54]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] =Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Clothing Store,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
1,"Agincourt, Agincourt",Clothing Store,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
2,"Birch Cliff, Cliffside West, Birch Cliff, Clif...",Skating Rink,General Entertainment,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,Gas Station,Fried Chicken Joint
3,"Cedarbrae, Cedarbrae",Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Caribbean Restaurant,Fried Chicken Joint,Construction & Landscaping,Hobby Shop
4,"Clairlea, Golden Mile, Oakridge, Clairlea, Gol...",Bus Line,Bakery,Park,Intersection,Bus Station,Ice Cream Shop,Soccer Field,Bar,Construction & Landscaping,Grocery Store


In [55]:
kclusters = 3
Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)
kmeans.labels_

array([1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [56]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Toronto_merged = Toronto_data
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Toronto_merged=Toronto_merged.drop([0, 16])
Toronto_merged.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union, Highla...",43.784535,-79.160497,0.0,Bar,Construction & Landscaping,Moving Target,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ...",43.763573,-79.188711,0.0,Intersection,Breakfast Spot,Rental Car Location,Electronics Store,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,College Stadium,General Entertainment
3,M1G,Scarborough,"Woburn, Woburn",43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Ice Cream Shop,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,M1H,Scarborough,"Cedarbrae, Cedarbrae",43.773136,-79.239476,0.0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Caribbean Restaurant,Fried Chicken Joint,Construction & Landscaping,Hobby Shop
5,M1J,Scarborough,"Scarborough Village, Scarborough Village",43.744734,-79.239476,1.0,Playground,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [57]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [58]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,0.0,Bar,Construction & Landscaping,Moving Target,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
2,Scarborough,0.0,Intersection,Breakfast Spot,Rental Car Location,Electronics Store,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,College Stadium,General Entertainment
3,Scarborough,0.0,Coffee Shop,Korean Restaurant,Ice Cream Shop,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,Scarborough,0.0,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Caribbean Restaurant,Fried Chicken Joint,Construction & Landscaping,Hobby Shop
6,Scarborough,0.0,Hobby Shop,Coffee Shop,Department Store,Chinese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
7,Scarborough,0.0,Bus Line,Bakery,Park,Intersection,Bus Station,Ice Cream Shop,Soccer Field,Bar,Construction & Landscaping,Grocery Store
9,Scarborough,0.0,Skating Rink,General Entertainment,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,Gas Station,Fried Chicken Joint
10,Scarborough,0.0,Indian Restaurant,Pet Store,Chinese Restaurant,Vietnamese Restaurant,Skating Rink,Clothing Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
11,Scarborough,0.0,Smoke Shop,Bakery,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
12,Scarborough,0.0,Clothing Store,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint


In [59]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,1.0,Playground,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
14,Scarborough,1.0,Park,Playground,Clothing Store,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
